In [1]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")

   Updating registry at `C:\Users\kgeri\.julia\registries\General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\kgeri\.julia\environments\v1.5\Manifest.toml`


In [1]:
using DataFrames
using CSV
using Printf
using Statistics
using LinearAlgebra

In [2]:
using PorousMaterials
using LightGraphs
using GraphPlot

In [15]:
pwd()

"C:\\Users\\kgeri\\Documents\\GitHub\\MolecularSynthesis"

In [18]:
atoms = read_xyz(joinpath(homedir(), "Documents", "GitHub", "MolecularSynthesis", "examples", "1.xyz"))

Atoms{Cart}(31, [:O, :C, :O, :C, :H, :C, :C, :C, :C, :C  …  :H, :H, :N, :C, :C, :H, :H, :O, :O, :H], Cart([1.428 0.344 … 1.716 0.039; 0.001 0.423 … 12.315 13.786; 0.64 -0.048 … -0.421 -0.14]))

In [19]:
box = Box(15.0,15.0,15.0)
atoms=Frac(atoms, box)

Atoms{Frac}(31, [:O, :C, :O, :C, :H, :C, :C, :C, :C, :C  …  :H, :H, :N, :C, :C, :H, :H, :O, :O, :H], Frac([0.09519999999999999 0.02293333333333333 … 0.11439999999999995 0.002599999999999944; 6.666666666666405e-5 0.0282 … 0.821 0.9190666666666666; 0.042666666666666665 -0.0032 … -0.028066666666666667 -0.009333333333333334]))

In [47]:
counter = 0
for i = 1:31
    counter += 1
    println(counter, " ", crystal.atoms.species[i])

end

1 O
2 C
3 O
4 C
5 H
6 C
7 C
8 C
9 C
10 C
11 C
12 C
13 Br
14 H
15 H
16 C
17 N
18 N
19 N
20 C
21 H
22 H
23 H
24 N
25 C
26 C
27 H
28 H
29 O
30 O
31 H


In [20]:
crystal = Crystal("NewTest.cif", box, atoms, Charges{Frac}(0))

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 31
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 9,:Br => 1,:O => 4,:C => 13)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [21]:
infer_bonds!(crystal, false)

In [22]:
Graph(crystal.bonds)
#Graph(edges(crystal.bonds))

{31, 32} undirected simple Int64 graph

In [23]:
#typeof(crystal.bonds)
#edges(crystal.bonds)

In [34]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to C
Edge 4 => 6
atom C is connected to C
Edge 4 => 10
atom C is connected to C
Edge 6 => 7
atom C is connected to C
Edge 7 => 8
atom C is connected to C
Edge 7 => 12
atom C is connected to C
Edge 8 => 9
atom C is connected to C
Edge 8 => 11
atom C is connected to C
Edge 9 => 10
atom C is connected to C
Edge 9 => 15
atom C is connected to H
Edge 10 => 14
atom C is connected to H
Edge 11 => 16
atom C is connected to C
Edge 11 => 19
atom C is connected to N
Edge 11 => 23
atom C is connected to H
Edge 12 => 24
atom C is connected to N
Edge 16 => 17
atom C is connected to N
Edge 16 => 21
atom C is connected to H
Edge 16 => 22
atom C is connected to H
Edge 17 => 18
atom N is connected to N
Edge 17 => 20
atom N is connected to C
Edge 18 => 19
atom N is connected to N
Edge 20 => 25
atom C is connected to C
Edge 20 => 28
atom C is connected to H
Edge 25

In [12]:
nbs = neighbors(crystal.bonds, 2)

crystal.atoms.species[21]

:C

In [13]:
### Function to identify carboxyl. When species is carbon, checks if it has two oxygen neighbors. 
### If it does, returns oxygen id's, anchor id, and oxygen's hydrogen id

function identify_carboxyl(crystal::Crystal, a::Int64)
   
    species = crystal.atoms.species[a]    #Identify species of selected atom
    nbs = neighbors(crystal.bonds, a)     #Identify neighbors of atom
    
    if species == :C                      #Identifying carboxylate starts with carbon
        oxygen_counter = 0
        oxygen_id = Int64[]
        X_id = 0
        H_id = 0
        for nb in nbs                     #Iterating through neighbors via identifying number
            species_nb = crystal.atoms.species[nb] #Get species of neighbor
            if species_nb == :O
                
                next_nbs = neighbors(crystal.bonds, nb)  #Get neighbors of oxygen (finding hydrogen)
                
                for next_nb in next_nbs
                    species_next_nb = crystal.atoms.species[next_nb]  #Get species of oxygen's neighbors
                    if species_next_nb == :H     #Identify hydrogen neighbor
                        H_id = next_nb
                    else
                    end
                end
                
                oxygen_counter += 1
                push!(oxygen_id, nb)         #Enter oxygen id into array
            else
                X_id = nb            #If neighbor of carboxylate carbon isn't oxygen, must be anchor
            end
        end
        
        if oxygen_counter == 2   #Assuming carboxylate carbon is only carbon with two oxygens bonded
        
            return true, oxygen_id, X_id, H_id
        else 
            return false, [], 0, 0
        end
    else
        return false, [], 0, 0
    end

end

identify_carboxyl (generic function with 1 method)

In [14]:
identify_carboxyl(crystal, 21)

(true, [26, 27], 28, 28)

In [119]:
#

keep = [true for i = 1:crystal.atoms.n]

X_species = Symbol[]
X_ids = Int64[]

for a = 1:crystal.atoms.n
       
   

    is_carboxyl, oxygen_ids, X_id, H_id = identify_carboxyl(crystal, a)
    
    if is_carboxyl == true 
    
        push!(X_species, crystal.atoms.species[X_id])
        push!(X_ids, X_id)
        crystal.atoms.species[X_id] = :X
        @assert length(oxygen_ids) == 2
        keep[oxygen_ids] .= false
        keep[a] = false
        keep[H_id] = false
        
    end


end

In [142]:
X_species
#X_ids

2-element Array{Symbol,1}:
 :C
 :H

In [121]:
for i in [19, 4]
    
   h = findfirst(isequal(i), X_ids) 
    print(h, "\n")
end

nothing
1


In [122]:
h = findfirst(isequal(19), X_ids) 
#X_species[h]

In [123]:
for ed in edges(crystal.bonds)
   
    println(ed)
    i = ed.src
    j = ed.dst
       
    species_i = crystal.atoms.species[i]
    species_j = crystal.atoms.species[j]
    println("atom ", species_i, " is connected to ", species_j)
    

end

Edge 1 => 2
atom O is connected to C
Edge 1 => 5
atom O is connected to H
Edge 2 => 3
atom C is connected to O
Edge 2 => 4
atom C is connected to X
Edge 4 => 6
atom X is connected to C
Edge 4 => 10
atom X is connected to C
Edge 6 => 7
atom C is connected to C
Edge 6 => 13
atom C is connected to H
Edge 7 => 8
atom C is connected to C
Edge 7 => 25
atom C is connected to O
Edge 8 => 9
atom C is connected to C
Edge 8 => 11
atom C is connected to C
Edge 9 => 10
atom C is connected to C
Edge 9 => 15
atom C is connected to H
Edge 10 => 14
atom C is connected to H
Edge 11 => 16
atom C is connected to C
Edge 12 => 22
atom C is connected to H
Edge 12 => 23
atom C is connected to H
Edge 12 => 24
atom C is connected to H
Edge 12 => 25
atom C is connected to O
Edge 16 => 17
atom C is connected to N
Edge 17 => 18
atom N is connected to N
Edge 18 => 19
atom N is connected to N
Edge 19 => 20
atom N is connected to N
Edge 20 => 21
atom N is connected to C
Edge 21 => 26
atom C is connected to O
Edge 21 

In [124]:
#Make a boolean array to keep atoms. 
crystal.atoms[keep]

Atoms{Frac}(20, [:X, :C, :C, :C, :C, :C, :C, :C, :H, :H, :H, :C, :N, :N, :N, :N, :H, :H, :H, :O], Frac([0.04639999999999999 -0.03226666666666668 … -0.2290666666666667 -0.11520000000000001; 0.13493333333333332 0.17600000000000002 … 0.36386666666666667 0.3091333333333333; 0.0016 0.018866666666666664 … 0.0021333333333333334 0.05093333333333333]))

In [125]:
tobacco_crystal = crystal.atoms[keep]

Atoms{Frac}(20, [:X, :C, :C, :C, :C, :C, :C, :C, :H, :H, :H, :C, :N, :N, :N, :N, :H, :H, :H, :O], Frac([0.04639999999999999 -0.03226666666666668 … -0.2290666666666667 -0.11520000000000001; 0.13493333333333332 0.17600000000000002 … 0.36386666666666667 0.3091333333333333; 0.0016 0.018866666666666664 … 0.0021333333333333334 0.05093333333333333]))

In [126]:
write_cif(crystal, "tobacco_crystal.cif")

In [127]:
crystal

Name: NewTest.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 28
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 7,:X => 2,:O => 5,:C => 10)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [128]:
write_cif(crystal, "tobacco_crystal.cif")

In [129]:
#infer_bonds!(tobacco_crystal, true)
#remove_bonds!(tobacco_crystal)

In [130]:
tobacco_crystal = Crystal("tobacco_crystal_2.cif", box, atoms[keep], Charges{Frac}(0))

Name: tobacco_crystal_2.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 20
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 6,:X => 1,:O => 1,:C => 8)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [131]:
function center_mass(crystal::Crystal)
    xf_cm = sum(crystal.atoms.coords.xf, dims = 2)
    frctn_xf_cm = xf_cm ./ crystal.atoms.n
    new_coords = crystal.atoms.coords.xf .- frctn_xf_cm
    crystal.atoms.coords.xf .= mod.(crystal.atoms.coords.xf, 1)
    crystal.atoms.coords.xf .= new_coords
    
    return crystal
end

center_mass (generic function with 1 method)

In [132]:
tobacco_crystal = center_mass(tobacco_crystal)

Name: tobacco_crystal_2.cif
Bravais unit cell of a crystal.
	Unit cell angles α = 90.000000 deg. β = 90.000000 deg. γ = 90.000000 deg.
	Unit cell dimensions a = 15.000000 Å. b = 15.000000 Å, c = 15.000000 Å
	Volume of unit cell: 3375.000000 Å³

	# atoms = 20
	# charges = 0
	chemical formula: Dict(:N => 4,:H => 6,:X => 1,:O => 1,:C => 8)
	space Group: P1
	symmetry Operations:
		'x, y, z'


In [133]:
tobacco_crystal.atoms

Atoms{Frac}(20, [:X, :C, :C, :C, :C, :C, :C, :C, :H, :H, :H, :C, :N, :N, :N, :N, :H, :H, :H, :O], Frac([0.06441333333333335 -0.014253333333333323 … -0.21105333333333334 -0.09718666666666666; -0.2172333333333333 -0.1761666666666666 … 0.011700000000000044 -0.04303333333333331; 0.0011066666666666655 0.01837333333333333 … 0.0016399999999999989 0.05044]))

In [134]:
"""
    write_cif(crystal, filename; fractional_coords=true, number_atoms=true)
Write a `crystal::Crystal` to a .cif file with `filename::AbstractString`. If `filename` does
not include the .cif extension, it will automatically be added. the `fractional_coords` flag
allows us to write either fractional or Cartesian coordinates.
"""
function write_cif_Kai(crystal::Crystal, filename::AbstractString, X_species::Array{Symbol,1}, X_ids::Array{Int64,1};
		 fractional_coords::Bool=true, number_atoms::Bool=true)
    if has_charges(crystal)
        if crystal.atoms.n != crystal.charges.n
            error("write_cif assumes equal numbers of Charges and Atoms (or zero charges)")
        end
        if ! isapprox(crystal.charges.coords, crystal.atoms.coords)
            error("write_cif needs coords of atoms and charges to correspond.")
        end
    end

    # TODO is this labeling necessary for the bonds, arthur?
    # create dictionary for tracking label numbers
    label_numbers = Dict{Symbol, Int}()
    for atom in crystal.atoms.species
        if !haskey(label_numbers, atom)
            label_numbers[atom] = 1
        end
    end

    # append ".cif" to filename if it doesn't already have the extension
    if ! occursin(".cif", filename)
        filename *= ".cif"
    end
    cif_file = open(filename, "w")
    # first line should be data_xtalname_PM
    if crystal.name == ""
        @printf(cif_file, "data_PM\n")
    else
        # don't include file extension!
        @printf(cif_file, "data_%s_PM\n", split(crystal.name, ".")[1])
    end

    @printf(cif_file, "_symmetry_space_group_name_H-M\t'%s'\n", crystal.symmetry.space_group)

    @printf(cif_file, "_cell_length_a\t%f\n", crystal.box.a)
    @printf(cif_file, "_cell_length_b\t%f\n", crystal.box.b)
    @printf(cif_file, "_cell_length_c\t%f\n", crystal.box.c)

    @printf(cif_file, "_cell_angle_alpha\t%f\n", crystal.box.α * 180.0 / pi)
    @printf(cif_file, "_cell_angle_beta\t%f\n", crystal.box.β * 180.0 / pi)
    @printf(cif_file, "_cell_angle_gamma\t%f\n", crystal.box.γ * 180.0 / pi)

    @printf(cif_file, "_symmetry_Int_Tables_number 1\n\n")
    @printf(cif_file, "loop_\n_symmetry_equiv_pos_as_xyz\n")
    for i in 1:size(crystal.symmetry.operations, 2)
        @printf(cif_file, "'%s,%s,%s'\n", crystal.symmetry.operations[:, i]...)
    end
    @printf(cif_file, "\n")

    @printf(cif_file, "loop_\n_atom_site_label\n_atom_site_type_symbol\n")
    if fractional_coords
        @printf(cif_file, "_atom_site_fract_x\n_atom_site_fract_y\n_atom_site_fract_z\n")
    else
        @printf(cif_file, "_atom_site_Cartn_x\n_atom_site_Cartn_y\n_atom_site_Cartn_z\n")
    end
    high_precision_charges = false # if, for neutrality, need high-precision charges
    if has_charges(crystal)
        @printf(cif_file, "_atom_site_charge\n")
        # if crystal will not be charge neutral to a 1e-5 tolerance when loading it
        #    into PorousMaterials.jl, then write higher-precision charges
        if abs(sum(round.(crystal.charges.q, digits=6))) > NET_CHARGE_TOL
            @info "writing high-precision charges for " * filename * ".\n"
            high_precision_charges = true
        end
    end

    idx_to_label = Array{AbstractString, 1}(undef, crystal.atoms.n)
    nb_of_X = 0
    for i = 1:crystal.atoms.n
        # print label and type symbol
        
        if crystal.atoms.species[i] == :X
            nb_of_X += 1
            #X_index = findfirst(isequal(i), X_ids)
            #println(X_index)
            element = X_species[nb_of_X]
            
            @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                (number_atoms ? string(label_numbers[crystal.atoms.species[i]]) : ""),
                element)
        else
            @printf(cif_file, "%s\t%s\t", string(crystal.atoms.species[i]) *
                (number_atoms ? string(label_numbers[crystal.atoms.species[i]]) : ""),
                crystal.atoms.species[i])
        end
        
        # store label for this atom idx
        idx_to_label[i] = string(crystal.atoms.species[i]) *
                    string(label_numbers[crystal.atoms.species[i]])
        # increment label
        label_numbers[crystal.atoms.species[i]] += 1
        if fractional_coords
            @printf(cif_file, "%f\t%f\t%f", crystal.atoms.coords.xf[:, i]...)
        else
            @printf(cif_file, "%f\t%f\t%f", (crystal.box.f_to_c * crystal.atoms.coords.xf[:, i])...)
        end
        if has_charges(crystal)
            if high_precision_charges
                @printf(cif_file, "\t%.10f\n", crystal.charges.q[i])
            else
                @printf(cif_file, "\t%f\n", crystal.charges.q[i])
            end
        else
            @printf(cif_file, "\n")
        end
    end

    # only print bond information if it is in the crystal
    if ne(crystal.bonds) > 0
        if ! number_atoms
             error("must label atoms with numbers to write bond information.\n")
        end
        # print column names for bond information
        @printf(cif_file, "\nloop_\n_geom_bond_atom_site_label_1\n_geom_bond_atom_site_label_2\n_geom_bond_distance\n_ccdc_geom_bond_type\n")

        for edge in collect(edges(crystal.bonds))
            dxf = crystal.atoms.coords.xf[:, edge.src] - crystal.atoms.coords.xf[:, edge.dst]
            nearest_image!(dxf)
            @printf(cif_file, "%s\t%s\t%0.5f\t%s\n", idx_to_label[edge.src], idx_to_label[edge.dst],
                    norm(dxf), ". A")
        end
    end
    close(cif_file)
end

write_cif_Kai

In [135]:
X_species

2-element Array{Symbol,1}:
 :C
 :H

In [136]:
X_ids

2-element Array{Int64,1}:
  4
 28

In [137]:
write_cif_Kai(tobacco_crystal, "tobacco_crystal_2.cif", X_species, X_ids)

In [138]:
infer_bonds!(tobacco_crystal, true)

In [141]:
write_cif_Kai(tobacco_crystal, "NewTest.cif", X_species, X_ids)